In [43]:
import pandas as pd

In [44]:
df = pd.read_parquet('../cache/merged.parquet')

In [45]:
df.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,GoldsteinScale,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h
0,2019-01-01,1546300800,None,None,CVL,None,None,None,None,Diplom Coop,3.4,4,50,2.354384,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464
1,2019-01-01,1546300800,BRA,BR,None,None,None,None,BR,Intent to Coop,4.0,3,30,-0.339542,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464
2,2019-01-01,1546300800,None,US,GOV,None,US,GOV,US,Appeal,3.2,10,40,1.734061,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464
3,2019-01-01,1546300800,USA,US,None,None,None,None,US,Disapprove,-2.0,3,21,-4.170072,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464
4,2019-01-01,1546300800,USA,US,EDU,None,None,None,US,Consult,1.9,6,18,1.415701,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464


In [46]:
quants = [0, 0.5, 0.9, 0.99]
mag_quantiles = [df['Magnitude'].quantile(quant) for quant in quants]

df['AbsChange'] = df['pct_change_30min'].abs()
avg_change = df.AbsChange.mean()
catg_features = ['Actor1Country', 'Actor1GeoCountry', 'Actor1Type',
                 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type',
                 'ActionCountry', 'EventType']

num_to_keep = 8
# Create dataframes for each quantile
dfs = [pd.DataFrame(df) for _ in range(len(quants))]

for feature in catg_features:
    for i, mag in enumerate(mag_quantiles):
        df_sub_orig = df[df.Magnitude > mag].astype(object)
        top_countries = df_sub_orig[feature].value_counts().index[:40]
        df_sub = df_sub_orig.groupby(feature).filter(lambda x: x.name in top_countries)
        df_grouped = (df_sub.groupby([feature], observed=True).AbsChange.mean() - avg_change).abs().sort_values(
            ascending=False)
        df_grouped = df_grouped[:num_to_keep]

        dfs[i][feature] = dfs[i][feature].fillna('Missing')
        dfs[i][feature] = dfs[i][feature].apply(lambda x: x if x in df_grouped.index else 'Other')
    print(f'Finished {feature}')

# Verification
for i, quant in enumerate(quants):
    print(f"\nFor quantile {quant}:")
    for feature in catg_features:
        print(f"{feature} unique values: {dfs[i][feature].nunique()}")

Finished Actor1Country
Finished Actor1GeoCountry
Finished Actor1Type
Finished Actor2Country
Finished Actor2GeoCountry
Finished Actor2Type
Finished ActionCountry
Finished EventType

For quantile 0:
Actor1Country unique values: 9
Actor1GeoCountry unique values: 9
Actor1Type unique values: 9
Actor2Country unique values: 9
Actor2GeoCountry unique values: 9
Actor2Type unique values: 9
ActionCountry unique values: 9
EventType unique values: 9

For quantile 0.5:
Actor1Country unique values: 9
Actor1GeoCountry unique values: 9
Actor1Type unique values: 9
Actor2Country unique values: 9
Actor2GeoCountry unique values: 9
Actor2Type unique values: 9
ActionCountry unique values: 9
EventType unique values: 9

For quantile 0.9:
Actor1Country unique values: 9
Actor1GeoCountry unique values: 9
Actor1Type unique values: 9
Actor2Country unique values: 9
Actor2GeoCountry unique values: 9
Actor2Type unique values: 9
ActionCountry unique values: 9
EventType unique values: 9

For quantile 0.99:
Actor1Country

In [47]:
merged_0q = dfs[0]  # 0 quantile (all data)
merged_50q = dfs[1]  # 0.5 quantile
merged_90q = dfs[2]  # 0.9 quantile
merged_99q = dfs[3]  # 0.99 quantile

In [48]:
merged_99q.shape

(5171541, 21)

In [49]:
# Convert EventType column to category dtype
merged_0q['EventType'] = merged_0q['EventType'].astype('category')

merged_0q.EventType.cat = merged_0q.EventType.cat.categories.append(pd.Index(['Missing', 'Other']))

In [50]:
merged_0q['EventType'].value_counts()

EventType
Other                4114248
Appeal                418795
Fight                 345688
Assault                81196
Demand                 69028
Reduce Relations       60881
Protest                59635
Demonstrate Force      20461
Mass Violence           1609
Name: count, dtype: int64

In [51]:
merged_99q['EventType'].value_counts()

EventType
Other                4393649
Yield                 202861
Aid                   174983
Reject                146931
Assault                81196
Threaten               80823
Demand                 69028
Demonstrate Force      20461
Mass Violence           1609
Name: count, dtype: int64

In [52]:
merged_0q['Actor1Type'].value_counts()

Actor1Type
Other    4872940
MIL       137301
HLH        68194
MNC        42983
UAF        34500
REB        13217
SET         1523
IMG          866
DEV           17
Name: count, dtype: int64

In [53]:
merged_99q['Actor1Type'].value_counts()

Actor1Type
Other    5141426
REF        11121
INS         7909
SEP         2444
UIS         2307
INT         1829
HRI         1748
SET         1523
ENV         1234
Name: count, dtype: int64

In [54]:
merged_99q['Actor1Type'].value_counts()

Actor1Type
Other    5141426
REF        11121
INS         7909
SEP         2444
UIS         2307
INT         1829
HRI         1748
SET         1523
ENV         1234
Name: count, dtype: int64

In [55]:
merged_50q.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,...,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h,AbsChange
0,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Diplom Coop,...,4,50,2.354384,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464,0.226363
1,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,3,30,-0.339542,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363
2,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,10,40,1.734061,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464,0.226363
3,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,3,21,-4.170072,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464,0.226363
4,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,6,18,1.415701,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363


In [56]:
merged_90q.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,...,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h,AbsChange
0,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Diplom Coop,...,4,50,2.354384,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464,0.226363
1,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Intent to Coop,...,3,30,-0.339542,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363
2,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,10,40,1.734061,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464,0.226363
3,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Disapprove,...,3,21,-4.170072,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464,0.226363
4,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,6,18,1.415701,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363


In [57]:
merged_99q.head()

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,...,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h,AbsChange
0,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,4,50,2.354384,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464,0.226363
1,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,3,30,-0.339542,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363
2,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,10,40,1.734061,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464,0.226363
3,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,3,21,-4.170072,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464,0.226363
4,2019-01-01,1546300800,Other,Other,Other,Other,Other,Other,Other,Other,...,6,18,1.415701,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464,0.226363


In [58]:
# keep only one row per timestamp with the least number of 'Other' values
def count_others(row):
    return sum(1 for x in row[catg_features] if x == 'Other')

def select_best(rows):
    min_others = rows['OtherCount'].min()
    best_rows = rows[rows['OtherCount'] == min_others]
    if len(best_rows) > 1:
        return best_rows.loc[best_rows['Magnitude'].idxmax()]
    return best_rows.iloc[0]

dfs = [merged_0q, merged_50q, merged_90q, merged_99q]
for i, df in enumerate(dfs):
    df['OtherCount'] = df.apply(count_others, axis=1)
    df = df.groupby('Timestamp').apply(lambda x: select_best(x))
    df.drop(columns='OtherCount', inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('Finished', i)
    print('Shape:', df.shape)
    dfs[i] = df
merged_0q, merged_50q, merged_90q, merged_99q = dfs

/var/folders/4n/lb57tn6n6c57_mmpd3t_9b1w0000gn/T/ipykernel_71212/3976218876.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Timestamp').apply(lambda x: select_best(x))


Finished 0
Shape: (171801, 21)


/var/folders/4n/lb57tn6n6c57_mmpd3t_9b1w0000gn/T/ipykernel_71212/3976218876.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Timestamp').apply(lambda x: select_best(x))


Finished 1
Shape: (171801, 21)


/var/folders/4n/lb57tn6n6c57_mmpd3t_9b1w0000gn/T/ipykernel_71212/3976218876.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Timestamp').apply(lambda x: select_best(x))


Finished 2
Shape: (171801, 21)
Finished 3
Shape: (171801, 21)


/var/folders/4n/lb57tn6n6c57_mmpd3t_9b1w0000gn/T/ipykernel_71212/3976218876.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Timestamp').apply(lambda x: select_best(x))


In [62]:
merged_0q.Actor1Type.value_counts()

Actor1Type
Other    137647
MIL       15444
HLH        6511
UAF        5326
MNC        3861
REB        2464
SET         451
IMG          97
Name: count, dtype: int64

In [63]:
merged_99q.Actor1Type.value_counts()

Actor1Type
Other    167791
REF        1514
INS         784
SET         496
UIS         329
SEP         293
INT         245
HRI         218
ENV         131
Name: count, dtype: int64

In [59]:
merged_0q.head(5)

,Date,Timestamp,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,...,NumSources,NumArticles,AvgTone,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h,AbsChange
0,2019-01-01 00:00:00,1546300800,Other,AS,MIL,Other,AS,Other,AS,Other,...,2,20,-9.007634,11.01,-7.71,Very Negative,-0.033061,-0.226363,-2.433464,0.226363
1,2019-01-01 00:15:00,1546301700,Other,US,Other,Other,Other,Other,US,Fight,...,8,80,-0.613089,8.61,-8.61,Very Negative,-0.193365,-2.578560,-1.891779,2.578560
2,2019-01-01 00:30:00,1546302600,Other,US,Other,Other,Other,Other,US,Fight,...,10,60,-5.068368,13.07,-13.07,Very Negative,-2.389815,-1.509301,-1.775994,1.509301
3,2019-01-01 00:45:00,1546303500,Other,AJ,Other,Other,Other,Other,AJ,Other,...,1,16,11.711712,13.01,4.42,Positive,0.902072,0.781614,0.895776,0.781614
4,2019-01-01 01:00:00,1546304400,Other,PE,UAF,Other,PE,Other,PE,Fight,...,2,20,-2.258977,4.26,-4.26,Negative,-0.119382,-0.291943,0.356518,0.291943


In [60]:
# check if all timestamps are present
print("Original:", df['Timestamp'].nunique())
print("0 quantile:", merged_0q['Timestamp'].nunique())
print("50 quantile:", merged_50q['Timestamp'].nunique())
print("90 quantile:", merged_90q['Timestamp'].nunique())
print("99 quantile:", merged_99q['Timestamp'].nunique())

Original: 171801
0 quantile: 171801
50 quantile: 171801
90 quantile: 171801
99 quantile: 171801


In [61]:
# Convert EventType column to category dtype
merged_0q['EventType'] = merged_0q['EventType'].astype('category')

merged_0q.to_parquet('../cache/merged_0q.parquet')
merged_50q.to_parquet('../cache/merged_50q.parquet')
merged_90q.to_parquet('../cache/merged_90q.parquet')
merged_99q.to_parquet('../cache/merged_99q.parquet')